# Classificação de Discurso de Ódio 
-------------------------------------
## Abordagem: tf-idf com Naive Bayes

-------------------------------------

## Importações e Definições

In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

from lime.lime_text import LimeTextExplainer

from utils import preprocess_text, format_lime_output, print_multilabel_metrics

In [66]:
target = ['aggressive', 'hate', 'ageism', 'aporophobia', 'body_shame', 'capacitism', 'lgbtphobia', 'political', 'racism', 'religious_intolerance', 'misogyny', 'xenophobia', 'other']
features = 'text'

In [67]:
# Configurações para o pré-processamento
config = {
    "lowercase": True,
    "remove_accents": True,
    "remove_punctuation": True,
    "remove_numbers": True,
    "remove_urls": True,
    "remove_mentions_hashtags": True,
    "expand_abbreviations": True,
    "expand_contractions": True,
    "normalize_laughter": True,
    "remove_emojis": False,
    "remove_stopwords": True,
    "lemmatize": True,
    "stemming": False,
    "pos_filter": False,
    "min_token_length": 2,
    "negation_scope": False,
    "replace_swears": False,
    "split_hashtags": False,
    "merge_mwes" : True,
    "replace_named_entities" : False
}

le = LabelEncoder()

--------------------------------------------
## Prepara o Conjunto de Dados

In [68]:
# 1. Carrega o dataset TuPyE multilabel
df = load_dataset("Silly-Machine/TuPyE-Dataset", name="multilabel")

train_df = df['train'].to_pandas()
test_df = df['test'].to_pandas()

X_train_raw = train_df[features]
train_df["label_comb"] = train_df[target].astype(str).agg("".join, axis=1)
train_df["class_id"] = le.fit_transform(train_df["label_comb"])
y_train = train_df["class_id"].values

X_test_raw = test_df[features]
y_test = test_df[target].values

### Aplica Pré-Processamento

In [69]:
X_train = X_train_raw.apply(lambda x: preprocess_text(x, config))
X_test = X_test_raw.apply(lambda x: preprocess_text(x, config))

--------------------------------------------
## Treinamento do Modelo

In [70]:
tfidf = TfidfVectorizer(
    max_features=10_000,       
    ngram_range=(1, 4),       
    min_df=2,                
    max_df=0.95,             
    sublinear_tf=True        
)

In [71]:
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [72]:
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


-----------------------------
## Define Explicador - LIME

In [73]:
def predict_proba(text_list):
    X_test = tfidf.transform(text_list)
    return model.predict_proba(X_test)

In [74]:
explainer = LimeTextExplainer(class_names=target)

### Teste com uma amostra

In [75]:
_texto = "vai tomar no seu cu seu viadinho de merda"
texto = preprocess_text(_texto, config)
exp = explainer.explain_instance(texto, predict_proba, num_features=6)

In [76]:
format_lime_output(texto, predict_proba, target, exp)

Categorias de discurso de ódio:
political(36%)

🔍 Explicação - presença das palavras:
cu(-0.017)


-----------------------------------
## Avaliação no conjunto de Teste

In [77]:
y_pred = model.predict(X_test_tfidf)

# converte saida inteira em multilabel de volta
multilabel_pred = [list(map(int, s)) for s in le.inverse_transform(y_pred)]

In [79]:
print_multilabel_metrics(y_test, multilabel_pred)


📊 Avaliação Multilabel
✔️ F1 Score (Micro):     0.0901
✔️ F1 Score (Macro):     0.0162
✔️ F1 Score (Weighted):  0.0855
⚠️ Hamming Loss:         0.0438
✅ Subset Accuracy:      0.7233


In [81]:
y_pred

array([0, 0, 0, ..., 0, 0, 0], shape=(8734,))

In [82]:
np.unique(y_pred, return_counts=True)

(array([ 0,  7, 13, 16, 31]), array([8472,  255,    5,    1,    1]))

In [84]:
y_train
np.unique(y_train, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]),
 array([24770,  1010,    41,    40,    28,    52,     1,  3923,   774,
            5,    46,    14,    28,  1290,    87,    35,   457,   280,
            3,     6,    56,    20,     2,     1,    81,    38,    22,
            9,    11,     7,     2,   750,     2,    17,    59,     1,
            1,     4,     3,     1,     1,   157,   123,     2,     1,
          239,    15,     1,     2,     2,     5,     2,     3,     1,
           13,     3,     1,    48,     3,     5,     1,     1,    16,
            4,     2,     1,    52,     2,   133,     3,     5,     8